In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

df = pd.read_csv(r"D:\Yebang\study\2024빅콘테스트\생성형AI분야\data\final_data\요약추가_12월_mct.csv")
df.head(4)

,기준연월,가맹점명,개설일자,업종,주소,이용건수구간,이용금액구간,건당평균이용금액구간,월_이용건수비중,화_이용건수비중,...,23-04시_이용건수비중,현지인_이용건수비중,남성_이용건수비중,여성_이용건수비중,10-20대_회원비중,30대_회원비중,40대_회원비중,50대_회원비중,60대이상_회원비중,요약
0,202312,협재해녀의집,20130627,가정식,제주 제주시 한림읍 협재리 1459-2번지,3_25~50%,3_25~50%,4_50~75%,0.000000,0.157407,...,0.0,0.020408,0.494,0.506,0.286,0.322,0.215,0.131,0.047,"협재해녀의집의 업종은 가정식, 위치는 제주 제주시 한림읍 협재리 1459-2번지 입..."
1,202312,카페보리다,20190717,커피,제주 제주시 이도이동 1055번지 1층,4_50~75%,5_75~90%,4_50~75%,0.060000,0.060000,...,0.0,0.788913,0.308,0.692,0.114,0.370,0.301,0.185,0.031,"카페보리다의 업종은 커피, 위치는 제주 제주시 이도이동 1055번지 1층 입니다.\..."
2,202312,영산강가마솥,20200217,가정식,제주 제주시 아라이동 1474-1번지 1층,4_50~75%,5_75~90%,4_50~75%,0.145833,0.145833,...,0.0,0.772894,0.608,0.392,0.015,0.105,0.145,0.364,0.370,"영산강가마솥의 업종은 가정식, 위치는 제주 제주시 아라이동 1474-1번지 1층 입..."
3,202312,범일분식,20151130,분식,제주 서귀포시 남원읍 남원리 139-7번지,3_25~50%,4_50~75%,5_75~90%,0.117117,0.135135,...,0.0,0.256170,0.604,0.396,0.183,0.350,0.240,0.155,0.071,"범일분식의 업종은 분식, 위치는 제주 서귀포시 남원읍 남원리 139-7번지 입니다...."


요약 컬럼 임베딩

In [4]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # 평균을 내어 문장 임베딩 생성
    return embeddings.squeeze().numpy()

# 요약 컬럼 임베딩
df['임베딩'] = df['요약'].apply(embed_text)


In [7]:
df['임베딩'][0]

array([-7.05683231e-01, -8.85592178e-02, -5.88768482e-01, -4.21409369e-01,
       -6.74269944e-02,  4.26992029e-01,  3.40804875e-01,  9.83347669e-02,
        3.06515008e-01, -2.91197538e-01,  5.69693387e-01, -6.46421835e-02,
        3.98718491e-02,  6.29209876e-02,  1.35569558e-01,  6.89093173e-01,
        2.92107731e-01,  6.78590909e-02, -2.58458048e-01,  3.18239868e-01,
        6.36847615e-02, -1.94532394e-01,  1.54702246e-01,  3.06408376e-01,
       -9.50372033e-03, -4.14349914e-01, -5.58160901e-01,  1.30334318e-01,
        3.60937297e-01,  4.22764689e-01,  3.84889990e-01, -7.13765547e-02,
       -2.52657294e-01, -1.50857061e-01, -1.44816875e-01,  2.08113790e-01,
       -6.18909836e-01, -5.31797051e-01,  1.47121405e-04,  5.50114457e-03,
       -8.25956985e-02,  9.60524157e-02, -1.02409020e-01,  2.38674581e-01,
        3.40870053e-01, -3.88115585e-01,  5.46302021e-01,  2.71414369e-01,
       -1.54803306e-01, -1.77113310e-01, -2.56520659e-01, -3.24607879e-01,
       -1.73837885e-01, -

In [13]:
import chromadb

# ChromaDB 클라이언트 초기화
client = chromadb.PersistentClient(path= r'D:\Yebang\study\2024빅콘테스트\생성형AI분야\ChromaDB2')

# Collection 생성 (이름은 원하는 대로 설정)
collection = client.create_collection("jeju_store_mct_v1")

# 데이터 저장
for index, row in df.iterrows():
    embedding = embed_text(row['요약'])  # 요약에 대한 임베딩 생성
    collection.add(
        ids=[f'doc_{index}'],  # 고유 ID 설정
        documents=[row['요약']],
        metadatas=[{'가맹점명': row['가맹점명']}],
        embeddings=[embedding]
    )


In [20]:
def search_restaurants(query):
    query_embedding = embed_text(query)  # 질문 임베딩
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5  # 반환할 결과 수
    )
    return results

# 예시 질문
query = "주말에 열지 않는 카페는?"
search_results = search_restaurants(query)

# 결과 출력
for result in search_results['metadatas'][0]:
    print(result['가맹점명'])

주식회사 제이콥에프앤비 무민카페제주점
카페방주
카페루시아(본점)
카페1950
카페구할구제주


In [25]:
print(df[df['가맹점명'] == '카페방주']['요약'])

2266    카페방주의 업종은 커피, 위치는 제주 서귀포시 안덕면 상천리 423-1번지 1층 입...
Name: 요약, dtype: object


In [26]:
print(df['요약'][2266])

카페방주의 업종은 커피, 위치는 제주 서귀포시 안덕면 상천리 423-1번지 1층 입니다.
이용건수는 동일 업종 내 상위 10~25%이고, 현지인 이용건수 비중은 0.07입니다.
이용건수가 가장 많은 요일은 일요일(0.24), 두 번째로 많은 요일은 수요일(0.14)입니다.
이용건수가 가장 많은 시간대는 12-13시(0.34), 두 번째로 많은 시간대는 14-17시(0.33)입니다.
여성의 이용건수가 더 높습니다. 남성 이용건수는 전체 가맹점 중 423위(0.43)이고, 여성 이용건수는 전체 가맹점 중 171위(0.57)입니다.
회원이 가장 많은 나이대는 50대(0.28)이고 전체 가맹점 중 197위입니다. 회원이 두 번째로 많은 나이대는 40대(0.26)이고 전체 가맹점 중 270위입니다.


In [29]:
import os
import chromadb
from transformers import AutoTokenizer, AutoModel
import torch

# 1. ChromaDB 데이터 저장 경로 설정 (이전에 저장했던 경로)
db_folder = r'D:\Yebang\study\2024빅콘테스트\생성형AI분야\ChromaDB2'

# 2. PersistentClient 사용하여 기존 데이터 로드
client = chromadb.PersistentClient(path=db_folder)

# 3. 기존에 생성한 컬렉션 로드
collection = client.get_collection("jeju_store_mct_v1")

# 4. 임베딩 모델 로드 (Hugging Face에서 모델 가져오기)
model_name = "jhgan/ko-sroberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 임베딩을 위한 함수 정의
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

def search_restaurants(query):
    query_embedding = embed_text(query)  # 질문 임베딩
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5  # 반환할 결과 수
    )
    return results

# 예시 질문
query = "주말에 열지 않는 카페는?"
search_results = search_restaurants(query)

# 결과 출력
for result in search_results['metadatas'][0]:
    print(result['가맹점명'])


주식회사 제이콥에프앤비 무민카페제주점
카페방주
카페루시아(본점)
카페1950
카페구할구제주
